In [102]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import shutil
import glob


# Get data from icdar 
80% training and 20% test

In [103]:
def dividing_data():
    
  files = os.listdir("our dataset/icdar_data/")
  X=[]
  for file in files:
    X.append("our dataset/icdar_data/"+file)

  train, test = train_test_split(X, test_size=0.2) 

  # delete all imgs in folder to replace it with a new data
  for i in os.listdir("our dataset/train_icdar/"):
    os.remove("our dataset/train_icdar/"+i)
  
  for i in os.listdir("our dataset/test_icdar/"):
    os.remove("our dataset/test_icdar/"+i)
  
    
  # save imgs to train and test folders 
  dst_dir = "our dataset/train_icdar/"
  for src_dir in train:
    for img in glob.iglob(src_dir):
        shutil.copy(img, dst_dir)
  
  dst_dir = "our dataset/test_icdar/"
  for src_dir in test:
    for img in glob.iglob(src_dir):
        shutil.copy(img, dst_dir)
  

In [104]:
def cropTxtOnly(image):
    img = image[30:-450,20:-20] # Perform pre-cropping

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray = 255*(gray < 90).astype(np.uint8) # To invert the text to white

    gray = 255*(gray < 160).astype(np.uint8) # To invert the text to white

    gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, np.ones((2, 2), dtype=np.uint8)) # Perform noise filtering

    coords = cv2.findNonZero(gray) # Find all non-zero points (text)
    x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
    rect = img[y:y+h, x:x+w] # Crop the image - note we do this on the original image

    return rect


In [112]:

def preprocessing(dataSrc,dataDest, target,icdar_or_dataset):
    
    files=[]
    genders=[]

    if icdar_or_dataset==1: #our dataset 
        genders=["/Female", "/Male"]
    else: 
        genders=[""]

    for gender in genders:
        files=os.listdir("our dataset/"+dataSrc+gender)
 
        for file in files:

            foldername=file.split('.')[0]
            if not os.path.exists("preprocessed/"+dataDest+gender+'/'+foldername):
                os.makedirs("preprocessed/"+dataDest+gender+'/'+foldername)

            if not os.path.exists(target):
                os.makedirs(target)

            # load image
            img = cv2.imread("our dataset/"+dataSrc + gender + "/" + file )
            #crop the text only from the image
            cropped_img=cropTxtOnly(img)
            
            if icdar_or_dataset==1:
                cv2.imwrite("preprocessed/"+dataDest+gender+'/'+foldername+'/'+"cropped.png", cropped_img) # Save the image

            # convert to gray
            gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

            # threshold the grayscale image
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

            # use morphology erode to blur horizontally
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (200, 3))
            morph = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)

            # use morphology open to remove thin lines from dotted lines
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 17))
            morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

            # find contours
            cntrs = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cntrs = cntrs[0] if len(cntrs) == 2 else cntrs[1]

            # if icdar_or_dataset==1:
            #     i=0
            #     thresh_result = thresh.copy()
            #     for c in cntrs:
            #         box = cv2.boundingRect(c)
            #         x,y,w,h = box
            #         cv2.imwrite("preprocessed/"+dataDest+gender+'/'+foldername + "/" + foldername + "-"+str(i)+".jpg", thresh_result[ y:y+h , x:x+w ] )
            #         i+=1

            # write result to disk
            cv2.imwrite(target+foldername+".jpg", thresh)



# Preprocessing training data

In [106]:
preprocessing("train","train","Training_data/",1)

# Preprocessing test data

In [113]:
preprocessing("test","test","Test_data/",1)

# Icdar data

In [108]:
dividing_data() #80% train and 20% test

In [110]:
preprocessing("test_icdar","test_icdar","Test_data/",0)
preprocessing("train_icdar","train_icdar","Training_data/",0)